In [ ]:
import os
import time

from argparse import Namespace
from tqdm.auto import tqdm
import random
import pickle

import itertools
from collections import defaultdict
import pandas as pd

import urllib.request
from bs4 import BeautifulSoup

from nltk.tokenize import RegexpTokenizer

In [ ]:
def get_fields(domain='cs'):
    fields = {}

    uh = urllib.request.urlopen(f'https://arxiv.org/archive/{domain}')
    soup = BeautifulSoup(str(uh.read()))

    for x in soup.findAll('ul')[2].findAll('li'):
        k = x.findAll('b')[0].text
        field, desc = k[3:5], k[8:]
        fields[field] = desc

    return fields

def get_abstract(url):
    uh = urllib.request.urlopen(url)
    soup = BeautifulSoup(str(uh.read()))
    a = soup.findAll('blockquote', {'class': 'abstract mathjax'})[0].text
    a = a.replace('\\n', ' ')
    a = a.replace('Abstract:  ', '')
    return a

# get_fields('cs')

In [ ]:
class Paper:
    def __init__(self, field, title, authors, pdf, abstract):
        self.field = field
        self.title = title
        self.authors = authors
        self.pdf = pdf
        self.abstract = abstract
        
    def __repr__(self):
        s = 'Paper {\n'
        s += ''.join([f'{k:>10}: {v}\n' for k, v in self.__dict__.items()])
        s += '}'
        return s

In [ ]:
CFG = Namespace()

CFG.url      = 'http://arxiv.org/list/cs.{}/{}{}?show={}'
CFG.fields   = ['AI', 'CL', 'CV', 'IR'][:1]
CFG.years    = ['{:0>2d}'.format(i) for i in range(17, 20)]
CFG.months   = ['{:0>2d}'.format(i) for i in range(1, 12)]
CFG.max_show = 1000

tokenizer = RegexpTokenizer(r'\w+')

In [ ]:
def crawl(fields: [str], years: [str], months: [str], max_show=1000) -> [Paper]:
    all_papers = []
    done = True
    
    info = list(itertools.product(fields, years, months))
    random.shuffle(info)
        
    for field, year, month in info:
        fname = f'out/papers-{field}-{year}-{month}.pkl'

        if os.path.exists(fname):
            continue
        
        done = False
        query_url = CFG.url.format(field, year, month, max_show)

        try:
            headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux i686) AppleWebKit/537.17 (KHTML, like Gecko) Chrome/24.0.1312.27 Safari/537.17'}
            req = urllib.request.Request(url=query_url, headers=headers) 
            uh = urllib.request.urlopen(req)
            soup = BeautifulSoup(str(uh.read()))
        except Exception as e:
            print(f'error for {query_url}: {e}')
            continue

        titles = soup.findAll('div', {'class': 'list-title'})
        authors = soup.findAll('div', {'class': 'list-authors'})
        urls = soup.findAll('span', {'class': 'list-identifier'})

        assert len(titles) == len(authors) == len(urls)

        papers = []
        for i in tqdm(range(len(titles)), desc=f'{field} | {year} | {month} '):
            t = titles[i].contents[-1].strip().replace('\\n', '')
            a = [x.string.strip() for x in authors[i].findAll('a')]
            ref = urls[i].contents[0].attrs['href'].split('/')[-1]
            p = f'http://arxiv.org/pdf/{ref}.pdf'
            abstract = get_abstract(url=f'http://arxiv.org/abs/{ref}')

            papers.append(Paper(field=field, title=t, authors=a, pdf=p, abstract=abstract))

        pickle.dump(papers, open(fname, 'wb'))
        all_papers += papers
        time.sleep(1*60)
        done = True

    return all_papers, done

In [ ]:
while True:
    _, done = crawl(CFG.fields, CFG.years, CFG.months, CFG.max_show)
    if done:
        break
    print('-' * 128)
    for i in tqdm(range(10), desc='sleep'):
        time.sleep(60)
        
print('Done')